In [1]:
import uproot
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import mplhep
plt.style.use(mplhep.style.LHCb2)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

In [2]:
def loadvars2df(rawdata, listofvars):
    dfs = []
    for i,data in enumerate(rawdata):
        dfi = data.arrays(listofvars, library='pd')
        #dfi = dfi.query('nuIdx<10000')
        dfi['uniqueid'] = dfi['nuIdx']+30000*i
        dfs.append(dfi)
    df = pd.concat(dfs, ignore_index=True)
    return df

nu_evt = []
nu_evt.append(uproot.open('/mnt/ssd/users/wenjiewu/flare_data/g4sim_ana_data/20221202_genie_v306_evt30000/nue_kling_ar40_genie_v306_evt30000.root:evt'))
nu_evt.append(uproot.open('/mnt/ssd/users/wenjiewu/flare_data/g4sim_ana_data/20221202_genie_v306_evt30000/numu_kling_ar40_genie_v306_evt30000.root:evt'))
nu_evt.append(uproot.open('/mnt/ssd/users/wenjiewu/flare_data/g4sim_ana_data/20221202_genie_v306_evt30000/nutau_bai_ar40_genie_v306_evt30000.root:evt'))
listofvars=['nuIdx', 'nuPDG', 'nuX', 'nuY', 'nuZ', 'nuE', 'nuIntType', 'nuScatteringType', 'nuFSLPDG', 'nuFSLPx', 'nuFSLPy', 'nuFSLPz', 'nuFSLE',
            'nPrimaryParticle', 'primaryTrackPDG', 'Px', 'Py', 'Pz', 'Pmass', 'VtxX', 'VtxY', 'VtxZ', 'primaryParentID', 'primaryTrackID', 'prongType', 'prongIndex',
            'primaryTrackLength', 'primaryTrackLengthInTPC', 'EInDetector', 'EInLAr', 'EInHadCal', 'EInMuonFinder', 'EInMuonFinderLayer1X', 'EInMuonFinderLayer1Y',
            'EInMuonFinderLayer2X', 'EInMuonFinderLayer2Y', 'AngleToBeamDir', 'ShowerLength', 'ShowerLengthInLAr', 'ShowerWidth', 'ShowerWidthInLAr', 
            'dEdx', 'dEdxInLAr', 'edepInLAr', 'edepInHadCalX', 'edepInHadCalY', 'edepInMuonFinderX', 'edepInMuonFinderY', 
            'edepInHadAborb', 'edepInMuonFinderAbsorb', 'edepInCryGap', 'nFromFSLParticles',
            'dir_pol_x', 'dir_pol_y', 'dir_pol_z', 'dir_coc_x', 'dir_coc_y', 'dir_coc_z']
df = loadvars2df(nu_evt, listofvars)

In [22]:
df.query('uniqueid==60000')[['uniqueid', 'prongIndex', 'nuPDG', 'primaryTrackPDG', 'prongType', 'EInLAr', 'primaryParentID']]

,uniqueid,prongIndex,nuPDG,primaryTrackPDG,prongType,EInLAr,primaryParentID
1265096,60000,0,16,-211,1,169.138452,0
1265097,60000,1,16,2212,1,17.184708,0
1265098,60000,2,16,2112,1,8.749791,0
1265099,60000,3,16,111,1,0.000000,0
1265100,60000,4,16,22,3,380.484548,8
1265101,60000,5,16,22,3,1308.641205,8
1265102,60000,6,16,2212,1,5361.905111,0
1265103,60000,7,16,-211,1,386.276734,0
1265104,60000,8,16,211,1,449.446342,0
1265105,60000,9,16,2112,1,0.000000,0


In [19]:
df.eval('TrueP = sqrt(Px**2 + Py**2 + Pz**2)', inplace=True)
df.eval('edep_tot = edepInLAr+edepInHadCalX+edepInHadCalY+edepInMuonFinderX+edepInMuonFinderY+edepInHadAborb+edepInMuonFinderAbsorb', inplace=True)
df.eval('EInLArFrac_prong = EInLAr/edepInLAr', inplace=True)

nProng = []
for event_id, df_event in df.groupby('uniqueid', sort=False):
    nProng.extend([df_event.shape[0]]*df_event.shape[0])
df['nProng'] = nProng

longestProng = []
longestProngIdx = []
leadingProng = []
leadingProngIdx = []
Pt = []
for event_id, df_event in df.groupby('uniqueid', sort=False):
    longestProng.extend([df_event['ShowerLength'].max()] * df_event.shape[0])
    longestProngIdx.extend([df.iloc[df_event['ShowerLength'].idxmax()]['prongIndex']] * df_event.shape[0])
    leadingProng.extend([df_event['EInDetector'].max()] * df_event.shape[0])
    leadingProngIdx.extend([df.iloc[df_event['EInDetector'].idxmax()]['prongIndex']] * df_event.shape[0])
    tot_px = np.sum(df_event.query('(primaryTrackPDG!=111) & (primaryTrackPDG!=15) & (EInLAr>0.1)')['Px'])
    tot_py = np.sum(df_event.query('(primaryTrackPDG!=111) & (primaryTrackPDG!=15) & (EInLAr>0.1)')['Py'])
    Pt.extend([np.sqrt(tot_px*tot_px+tot_py*tot_py)/1000] * df_event.shape[0])
df['longestProng'] = longestProng
df['longestProngIdx'] = longestProngIdx
df['leadingProng'] = leadingProng
df['leadingProngIdx'] = leadingProngIdx
df['Pt'] = Pt
df['PtP'] = df['Pt']/df['nuE']
df['PtEdep'] = df['Pt']/df['edep_tot']*1000

In [23]:
num_piminus = []; num_piplus = []; num_pi0 = []; num_proton = []; num_neutron = []; num_kplus = []; num_kminus = [];
num_gamma = []; num_nu = []; num_muon = [];

leading_piminus_momentum = []; leading_piminus_showerlength = [];
leading_piminus_edep = []; leading_piminus_idx = [];
angle_to_leading_piminus = [];
first_closest_shower_to_leading_piminus_angle = []
first_closest_shower_to_leading_piminus_showerlength = []
first_closest_shower_to_leading_piminus_showerwidth = []
first_closest_shower_to_leading_piminus_edep = []
second_closest_shower_to_leading_piminus_angle = []
second_closest_shower_to_leading_piminus_showerlength = []
second_closest_shower_to_leading_piminus_showerwidth = []
second_closest_shower_to_leading_piminus_edep = []
third_closest_shower_to_leading_piminus_angle = []
third_closest_shower_to_leading_piminus_showerlength = []
third_closest_shower_to_leading_piminus_showerwidth = []
third_closest_shower_to_leading_piminus_edep = []
fourth_closest_shower_to_leading_piminus_angle = []
fourth_closest_shower_to_leading_piminus_showerlength = []
fourth_closest_shower_to_leading_piminus_showerwidth = []
fourth_closest_shower_to_leading_piminus_edep = []

for event_id, df_event in df.groupby('uniqueid', sort=False):
    if event_id%5000==0:
        print("Processing ", event_id)
    num_pi0.extend([np.sum(df_event['primaryTrackPDG']==111)] * df_event.shape[0])
    num_piplus.extend([np.sum(df_event['primaryTrackPDG']==211)] * df_event.shape[0])
    num_piminus.extend([np.sum(df_event['primaryTrackPDG']==-211)] * df_event.shape[0])
    num_proton.extend([np.sum(df_event['primaryTrackPDG']==2212)] * df_event.shape[0])
    num_neutron.extend([np.sum(df_event['primaryTrackPDG']==2112)] * df_event.shape[0])
    num_gamma.extend([np.sum(df_event['primaryTrackPDG']==22)] * df_event.shape[0])
    num_nu.extend([np.sum((abs(df_event['primaryTrackPDG'])==16) | (abs(df_event['primaryTrackPDG'])==14) | (abs(df_event['primaryTrackPDG'])==12))] * df_event.shape[0])
    num_kplus.extend([np.sum(df_event['primaryTrackPDG']==321)] * df_event.shape[0])
    num_kminus.extend([np.sum(df_event['primaryTrackPDG']==-321)] * df_event.shape[0])
    num_muon.extend([np.sum(df_event['primaryTrackPDG']==13)] * df_event.shape[0])
    
    _leading_piminus_momentum = -1; _leading_piminus_showerlength = -1; _leading_piminus_edep = -1; _leading_piminus_idx = -1; 
    if np.sum(df_event['primaryTrackPDG']==-211) > 0:
        _leading_piminus_momentum = df_event[df_event['primaryTrackPDG']==-211]['TrueP'].max()/1000
        _leading_piminus_idx = df.iloc[df_event[df_event['primaryTrackPDG']==-211]['TrueP'].idxmax()]['prongIndex']
        _leading_piminus_showerlength = df.iloc[df_event[df_event['primaryTrackPDG']==-211]['TrueP'].idxmax()]['ShowerLength']
        _leading_piminus_edep         = df.iloc[df_event[df_event['primaryTrackPDG']==-211]['TrueP'].idxmax()]['EInDetector']
    leading_piminus_momentum.extend([_leading_piminus_momentum] * df_event.shape[0])
    leading_piminus_idx.extend([_leading_piminus_idx] * df_event.shape[0])
    leading_piminus_showerlength.extend([_leading_piminus_showerlength] * df_event.shape[0])
    leading_piminus_edep.extend([_leading_piminus_edep] * df_event.shape[0])
    
    _angle_to_leading_piminus = []
    if np.sum(df_event['primaryTrackPDG']==-211) == 0:
        _angle_to_leading_piminus.extend([-1] * df_event.shape[0])
    else:
        _leading_piminus_vecP = df_event[df_event['prongIndex']==_leading_piminus_idx][['Px', 'Py', 'Pz']].to_numpy()[0]
        for particle_id, df_particle in df_event.groupby('prongIndex', sort=False):
            if df_particle['prongIndex'].values[0]==_leading_piminus_idx:
                _angle_to_leading_piminus.append(0)
            else:
                _angle_to_leading_piminus.append(np.arccos(min(np.sum(df_particle[['Px', 'Py', 'Pz']].to_numpy()[0] * _leading_piminus_vecP) \
                                /_leading_piminus_momentum/1000/df_particle['TrueP'].values[0],1))*180/np.pi)
    angle_to_leading_piminus.extend(_angle_to_leading_piminus)
    
    max_closest_idx = 4; closest_idx = 0;
    _first_closest_shower_to_leading_piminus_angle = -1
    _first_closest_shower_to_leading_piminus_showerlength = -1
    _first_closest_shower_to_leading_piminus_showerwidth = -1
    _first_closest_shower_to_leading_piminus_edep = -1
    _second_closest_shower_to_leading_piminus_angle = -1
    _second_closest_shower_to_leading_piminus_showerlength = -1
    _second_closest_shower_to_leading_piminus_showerwidth = -1
    _second_closest_shower_to_leading_piminus_edep = -1
    _third_closest_shower_to_leading_piminus_angle = -1
    _third_closest_shower_to_leading_piminus_showerlength = -1
    _third_closest_shower_to_leading_piminus_showerwidth = -1
    _third_closest_shower_to_leading_piminus_edep = -1
    _fourth_closest_shower_to_leading_piminus_angle = -1
    _fourth_closest_shower_to_leading_piminus_showerlength = -1
    _fourth_closest_shower_to_leading_piminus_showerwidth = -1
    _fourth_closest_shower_to_leading_piminus_edep = -1
    for _angle, _prongIdx in sorted(zip(_angle_to_leading_piminus, df_event['prongIndex'].to_numpy())):
        #if abs(df_event[df_event['prongIndex']==_prongIdx]['primaryTrackPDG'].values[0]) in [12,14,15,16,2112] or abs(df_event[df_event['prongIdx']==_prongIdx]['primaryTrackPDG'].values[0])>3000:
        #    continue
        if abs(df_event[df_event['prongIndex']==_prongIdx]['primaryTrackPDG'].values[0]) not in [11, 22]:
            continue
        if _angle > 0 and closest_idx < max_closest_idx:
            if closest_idx == 0:
                _first_closest_shower_to_leading_piminus_angle = _angle
                _first_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _first_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _first_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            if closest_idx == 1:
                _second_closest_shower_to_leading_piminus_angle = _angle
                _second_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _second_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _second_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            if closest_idx == 2:
                _third_closest_shower_to_leading_piminus_angle = _angle
                _third_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _third_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _third_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            if closest_idx == 3:
                _fourth_closest_shower_to_leading_piminus_angle = _angle
                _fourth_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _fourth_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _fourth_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            closest_idx += 1
    first_closest_shower_to_leading_piminus_angle.extend([_first_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    first_closest_shower_to_leading_piminus_showerlength.extend([_first_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    first_closest_shower_to_leading_piminus_showerwidth.extend([_first_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    first_closest_shower_to_leading_piminus_edep.extend([_first_closest_shower_to_leading_piminus_edep] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_angle.extend([_second_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_showerlength.extend([_second_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_showerwidth.extend([_second_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_edep.extend([_second_closest_shower_to_leading_piminus_edep] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_angle.extend([_third_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_showerlength.extend([_third_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_showerwidth.extend([_third_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_edep.extend([_third_closest_shower_to_leading_piminus_edep] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_angle.extend([_fourth_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_showerlength.extend([_fourth_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_showerwidth.extend([_fourth_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_edep.extend([_fourth_closest_shower_to_leading_piminus_edep] * df_event.shape[0])

Processing  0
Processing  10000
Processing  15000
Processing  20000
Processing  25000
Processing  5000
Processing  30000
Processing  40000
Processing  45000
Processing  50000
Processing  55000
Processing  35000
Processing  60000
Processing  70000
Processing  75000
Processing  80000
Processing  85000
Processing  65000


In [24]:
df['num_piminus'] = num_piminus
df['num_piplus']  = num_piplus
df['num_pi0']     = num_pi0
df['num_proton']  = num_proton
df['num_neutron'] = num_neutron
df['num_kplus']   = num_kplus
df['num_kminus']  = num_kminus
df['num_gamma']   = num_gamma
df['num_nu']      = num_nu
df['num_muon']    = num_muon
df['leading_piminus_momentum'] = leading_piminus_momentum
df['leading_piminus_idx'] = leading_piminus_idx
df['leading_piminus_showerlength'] = leading_piminus_showerlength
df['leading_piminus_edep'] = leading_piminus_edep
df['angle_to_leading_piminus'] = angle_to_leading_piminus
df['first_closest_shower_to_leading_piminus_angle'] = first_closest_shower_to_leading_piminus_angle
df['first_closest_shower_to_leading_piminus_showerlength'] = first_closest_shower_to_leading_piminus_showerlength
df['first_closest_shower_to_leading_piminus_showerwidth'] = first_closest_shower_to_leading_piminus_showerwidth
df['first_closest_shower_to_leading_piminus_edep'] = first_closest_shower_to_leading_piminus_edep
df['second_closest_shower_to_leading_piminus_angle'] = second_closest_shower_to_leading_piminus_angle
df['second_closest_shower_to_leading_piminus_showerlength'] = second_closest_shower_to_leading_piminus_showerlength
df['second_closest_shower_to_leading_piminus_showerwidth'] = second_closest_shower_to_leading_piminus_showerwidth
df['second_closest_shower_to_leading_piminus_edep'] = second_closest_shower_to_leading_piminus_edep
df['third_closest_shower_to_leading_piminus_angle'] = third_closest_shower_to_leading_piminus_angle
df['third_closest_shower_to_leading_piminus_showerlength'] = third_closest_shower_to_leading_piminus_showerlength
df['third_closest_shower_to_leading_piminus_showerwidth'] = third_closest_shower_to_leading_piminus_showerwidth
df['third_closest_shower_to_leading_piminus_edep'] = third_closest_shower_to_leading_piminus_edep
df['fourth_closest_shower_to_leading_piminus_angle'] = fourth_closest_shower_to_leading_piminus_angle
df['fourth_closest_shower_to_leading_piminus_showerlength'] = fourth_closest_shower_to_leading_piminus_showerlength
df['fourth_closest_shower_to_leading_piminus_showerwidth'] = fourth_closest_shower_to_leading_piminus_showerwidth
df['fourth_closest_shower_to_leading_piminus_edep'] = fourth_closest_shower_to_leading_piminus_edep

del num_piminus; del num_piplus; del num_pi0; del num_proton; del num_neutron; del num_kplus; del num_kminus; del num_gamma; del num_nu; del num_muon;
del leading_piminus_momentum; del leading_piminus_idx; del leading_piminus_showerlength; del leading_piminus_edep; del angle_to_leading_piminus;
del first_closest_shower_to_leading_piminus_angle; del first_closest_shower_to_leading_piminus_showerlength; del first_closest_shower_to_leading_piminus_edep; del first_closest_shower_to_leading_piminus_showerwidth;
del second_closest_shower_to_leading_piminus_angle; del second_closest_shower_to_leading_piminus_showerlength; del second_closest_shower_to_leading_piminus_edep; del second_closest_shower_to_leading_piminus_showerwidth;
del third_closest_shower_to_leading_piminus_angle; del third_closest_shower_to_leading_piminus_showerlength; del third_closest_shower_to_leading_piminus_edep; del third_closest_shower_to_leading_piminus_showerwidth;
del fourth_closest_shower_to_leading_piminus_angle; del fourth_closest_shower_to_leading_piminus_showerlength; del fourth_closest_shower_to_leading_piminus_edep; del fourth_closest_shower_to_leading_piminus_showerwidth;

In [25]:
df.to_pickle("/mnt/ssd/users/wenjiewu/flare_data/g4sim_ana_data/20221202_genie_v306_evt30000/features_geniev306_trueDirection.pkl")  

In [12]:
df.query('primaryParentID==1 and nuPDG!=16')[['nuIdx', 'uniqueid', 'nuPDG', 'nuE', 'primaryTrackPDG', 'prongType']]

,nuIdx,uniqueid,nuPDG,nuE,primaryTrackPDG,prongType
874827,7486,37486,14,122.617448,11,9
874828,7486,37486,14,122.617448,-12,9
874829,7486,37486,14,122.617448,14,9


In [26]:
num_piminus = []; num_piplus = []; num_pi0 = []; num_proton = []; num_neutron = []; num_kplus = []; num_kminus = [];
num_gamma = []; num_nu = []; num_muon = [];

leading_piminus_momentum = []; leading_piminus_showerlength = [];
leading_piminus_edep = []; leading_piminus_idx = [];
angle_to_leading_piminus = [];
first_closest_shower_to_leading_piminus_angle = []
first_closest_shower_to_leading_piminus_showerlength = []
first_closest_shower_to_leading_piminus_showerwidth = []
first_closest_shower_to_leading_piminus_edep = []
second_closest_shower_to_leading_piminus_angle = []
second_closest_shower_to_leading_piminus_showerlength = []
second_closest_shower_to_leading_piminus_showerwidth = []
second_closest_shower_to_leading_piminus_edep = []
third_closest_shower_to_leading_piminus_angle = []
third_closest_shower_to_leading_piminus_showerlength = []
third_closest_shower_to_leading_piminus_showerwidth = []
third_closest_shower_to_leading_piminus_edep = []
fourth_closest_shower_to_leading_piminus_angle = []
fourth_closest_shower_to_leading_piminus_showerlength = []
fourth_closest_shower_to_leading_piminus_showerwidth = []
fourth_closest_shower_to_leading_piminus_edep = []

for event_id, df_event in df.groupby('uniqueid', sort=False):
    if event_id%5000==0:
        print("Processing ", event_id)
    num_pi0.extend([np.sum(df_event['primaryTrackPDG']==111)] * df_event.shape[0])
    num_piplus.extend([np.sum(df_event['primaryTrackPDG']==211)] * df_event.shape[0])
    num_piminus.extend([np.sum(df_event['primaryTrackPDG']==-211)] * df_event.shape[0])
    num_proton.extend([np.sum(df_event['primaryTrackPDG']==2212)] * df_event.shape[0])
    num_neutron.extend([np.sum(df_event['primaryTrackPDG']==2112)] * df_event.shape[0])
    num_gamma.extend([np.sum(df_event['primaryTrackPDG']==22)] * df_event.shape[0])
    num_nu.extend([np.sum((abs(df_event['primaryTrackPDG'])==16) | (abs(df_event['primaryTrackPDG'])==14) | (abs(df_event['primaryTrackPDG'])==12))] * df_event.shape[0])
    num_kplus.extend([np.sum(df_event['primaryTrackPDG']==321)] * df_event.shape[0])
    num_kminus.extend([np.sum(df_event['primaryTrackPDG']==-321)] * df_event.shape[0])
    num_muon.extend([np.sum(df_event['primaryTrackPDG']==13)] * df_event.shape[0])
    
    _leading_piminus_momentum = -1; _leading_piminus_showerlength = -1; _leading_piminus_edep = -1; _leading_piminus_idx = -1; 
    if np.sum(df_event['primaryTrackPDG']==-211) > 0:
        _leading_piminus_momentum = df_event[df_event['primaryTrackPDG']==-211]['TrueP'].max()/1000
        _leading_piminus_idx = df.iloc[df_event[df_event['primaryTrackPDG']==-211]['TrueP'].idxmax()]['prongIndex']
        _leading_piminus_showerlength = df.iloc[df_event[df_event['primaryTrackPDG']==-211]['TrueP'].idxmax()]['ShowerLength']
        _leading_piminus_edep         = df.iloc[df_event[df_event['primaryTrackPDG']==-211]['TrueP'].idxmax()]['EInDetector']
    leading_piminus_momentum.extend([_leading_piminus_momentum] * df_event.shape[0])
    leading_piminus_idx.extend([_leading_piminus_idx] * df_event.shape[0])
    leading_piminus_showerlength.extend([_leading_piminus_showerlength] * df_event.shape[0])
    leading_piminus_edep.extend([_leading_piminus_edep] * df_event.shape[0])
    
    _angle_to_leading_piminus = []
    if np.sum(df_event['primaryTrackPDG']==-211) == 0:
        _angle_to_leading_piminus.extend([-1] * df_event.shape[0])
    else:
        _leading_piminus_vecP = df_event[df_event['prongIndex']==_leading_piminus_idx][['dir_coc_x', 'dir_coc_y', 'dir_coc_z']].to_numpy()[0]
        for particle_id, df_particle in df_event.groupby('prongIndex', sort=False):
            if df_particle['prongIndex'].values[0]==_leading_piminus_idx:
                _angle_to_leading_piminus.append(0)
            else:
                _angle_to_leading_piminus.append(np.arccos(min(np.sum(df_particle[['dir_coc_x', 'dir_coc_y', 'dir_coc_z']].to_numpy()[0] * _leading_piminus_vecP) \
                                /_leading_piminus_momentum/1000/df_particle['TrueP'].values[0],1))*180/np.pi)
    angle_to_leading_piminus.extend(_angle_to_leading_piminus)
    
    max_closest_idx = 4; closest_idx = 0;
    _first_closest_shower_to_leading_piminus_angle = -1
    _first_closest_shower_to_leading_piminus_showerlength = -1
    _first_closest_shower_to_leading_piminus_showerwidth = -1
    _first_closest_shower_to_leading_piminus_edep = -1
    _second_closest_shower_to_leading_piminus_angle = -1
    _second_closest_shower_to_leading_piminus_showerlength = -1
    _second_closest_shower_to_leading_piminus_showerwidth = -1
    _second_closest_shower_to_leading_piminus_edep = -1
    _third_closest_shower_to_leading_piminus_angle = -1
    _third_closest_shower_to_leading_piminus_showerlength = -1
    _third_closest_shower_to_leading_piminus_showerwidth = -1
    _third_closest_shower_to_leading_piminus_edep = -1
    _fourth_closest_shower_to_leading_piminus_angle = -1
    _fourth_closest_shower_to_leading_piminus_showerlength = -1
    _fourth_closest_shower_to_leading_piminus_showerwidth = -1
    _fourth_closest_shower_to_leading_piminus_edep = -1
    for _angle, _prongIdx in sorted(zip(_angle_to_leading_piminus, df_event['prongIndex'].to_numpy())):
        #if abs(df_event[df_event['prongIndex']==_prongIdx]['primaryTrackPDG'].values[0]) in [12,14,15,16,2112] or abs(df_event[df_event['prongIdx']==_prongIdx]['primaryTrackPDG'].values[0])>3000:
        #    continue
        if abs(df_event[df_event['prongIndex']==_prongIdx]['primaryTrackPDG'].values[0]) not in [11, 22]:
            continue
        if _angle > 0 and closest_idx < max_closest_idx:
            if closest_idx == 0:
                _first_closest_shower_to_leading_piminus_angle = _angle
                _first_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _first_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _first_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            if closest_idx == 1:
                _second_closest_shower_to_leading_piminus_angle = _angle
                _second_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _second_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _second_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            if closest_idx == 2:
                _third_closest_shower_to_leading_piminus_angle = _angle
                _third_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _third_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _third_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            if closest_idx == 3:
                _fourth_closest_shower_to_leading_piminus_angle = _angle
                _fourth_closest_shower_to_leading_piminus_showerlength = df_event[df_event['prongIndex']==_prongIdx]['ShowerLength'].values[0]
                _fourth_closest_shower_to_leading_piminus_showerwidth = df_event[df_event['prongIndex']==_prongIdx]['ShowerWidthInLAr'].values[0]
                _fourth_closest_shower_to_leading_piminus_edep = df_event[df_event['prongIndex']==_prongIdx]['EInDetector'].values[0]
            closest_idx += 1
    first_closest_shower_to_leading_piminus_angle.extend([_first_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    first_closest_shower_to_leading_piminus_showerlength.extend([_first_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    first_closest_shower_to_leading_piminus_showerwidth.extend([_first_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    first_closest_shower_to_leading_piminus_edep.extend([_first_closest_shower_to_leading_piminus_edep] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_angle.extend([_second_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_showerlength.extend([_second_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_showerwidth.extend([_second_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    second_closest_shower_to_leading_piminus_edep.extend([_second_closest_shower_to_leading_piminus_edep] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_angle.extend([_third_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_showerlength.extend([_third_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_showerwidth.extend([_third_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    third_closest_shower_to_leading_piminus_edep.extend([_third_closest_shower_to_leading_piminus_edep] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_angle.extend([_fourth_closest_shower_to_leading_piminus_angle] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_showerlength.extend([_fourth_closest_shower_to_leading_piminus_showerlength] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_showerwidth.extend([_fourth_closest_shower_to_leading_piminus_showerwidth] * df_event.shape[0])
    fourth_closest_shower_to_leading_piminus_edep.extend([_fourth_closest_shower_to_leading_piminus_edep] * df_event.shape[0])

Processing  0
Processing  10000
Processing  15000
Processing  20000
Processing  25000
Processing  5000
Processing  30000
Processing  40000
Processing  45000
Processing  50000
Processing  55000
Processing  35000
Processing  60000
Processing  70000
Processing  75000
Processing  80000
Processing  85000
Processing  65000


In [27]:
df['num_piminus'] = num_piminus
df['num_piplus']  = num_piplus
df['num_pi0']     = num_pi0
df['num_proton']  = num_proton
df['num_neutron'] = num_neutron
df['num_kplus']   = num_kplus
df['num_kminus']  = num_kminus
df['num_gamma']   = num_gamma
df['num_nu']      = num_nu
df['num_muon']    = num_muon
df['leading_piminus_momentum'] = leading_piminus_momentum
df['leading_piminus_idx'] = leading_piminus_idx
df['leading_piminus_showerlength'] = leading_piminus_showerlength
df['leading_piminus_edep'] = leading_piminus_edep
df['angle_to_leading_piminus'] = angle_to_leading_piminus
df['first_closest_shower_to_leading_piminus_angle'] = first_closest_shower_to_leading_piminus_angle
df['first_closest_shower_to_leading_piminus_showerlength'] = first_closest_shower_to_leading_piminus_showerlength
df['first_closest_shower_to_leading_piminus_showerwidth'] = first_closest_shower_to_leading_piminus_showerwidth
df['first_closest_shower_to_leading_piminus_edep'] = first_closest_shower_to_leading_piminus_edep
df['second_closest_shower_to_leading_piminus_angle'] = second_closest_shower_to_leading_piminus_angle
df['second_closest_shower_to_leading_piminus_showerlength'] = second_closest_shower_to_leading_piminus_showerlength
df['second_closest_shower_to_leading_piminus_showerwidth'] = second_closest_shower_to_leading_piminus_showerwidth
df['second_closest_shower_to_leading_piminus_edep'] = second_closest_shower_to_leading_piminus_edep
df['third_closest_shower_to_leading_piminus_angle'] = third_closest_shower_to_leading_piminus_angle
df['third_closest_shower_to_leading_piminus_showerlength'] = third_closest_shower_to_leading_piminus_showerlength
df['third_closest_shower_to_leading_piminus_showerwidth'] = third_closest_shower_to_leading_piminus_showerwidth
df['third_closest_shower_to_leading_piminus_edep'] = third_closest_shower_to_leading_piminus_edep
df['fourth_closest_shower_to_leading_piminus_angle'] = fourth_closest_shower_to_leading_piminus_angle
df['fourth_closest_shower_to_leading_piminus_showerlength'] = fourth_closest_shower_to_leading_piminus_showerlength
df['fourth_closest_shower_to_leading_piminus_showerwidth'] = fourth_closest_shower_to_leading_piminus_showerwidth
df['fourth_closest_shower_to_leading_piminus_edep'] = fourth_closest_shower_to_leading_piminus_edep

del num_piminus; del num_piplus; del num_pi0; del num_proton; del num_neutron; del num_kplus; del num_kminus; del num_gamma; del num_nu; del num_muon;
del leading_piminus_momentum; del leading_piminus_idx; del leading_piminus_showerlength; del leading_piminus_edep; del angle_to_leading_piminus;
del first_closest_shower_to_leading_piminus_angle; del first_closest_shower_to_leading_piminus_showerlength; del first_closest_shower_to_leading_piminus_edep; del first_closest_shower_to_leading_piminus_showerwidth;
del second_closest_shower_to_leading_piminus_angle; del second_closest_shower_to_leading_piminus_showerlength; del second_closest_shower_to_leading_piminus_edep; del second_closest_shower_to_leading_piminus_showerwidth;
del third_closest_shower_to_leading_piminus_angle; del third_closest_shower_to_leading_piminus_showerlength; del third_closest_shower_to_leading_piminus_edep; del third_closest_shower_to_leading_piminus_showerwidth;
del fourth_closest_shower_to_leading_piminus_angle; del fourth_closest_shower_to_leading_piminus_showerlength; del fourth_closest_shower_to_leading_piminus_edep; del fourth_closest_shower_to_leading_piminus_showerwidth;

In [28]:
df.to_pickle("/mnt/ssd/users/wenjiewu/flare_data/g4sim_ana_data/20221202_genie_v306_evt30000/features_geniev306_COCDir.pkl")  